<a href="https://colab.research.google.com/github/JSunkel/Forecasting-Models/blob/master/AAL_all_devices_Prophet_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from fbprophet import Prophet

In [0]:
# Load the full dataset
fact_table = pd.read_csv('drive/My Drive/AAL/aal_fact_table.csv')

In [4]:
# Check the full dataset format (testing only)
fact_table.head()

,ds,phone,tablet,wearable,others
0,2017-01-01,3558,4925,242,414
1,2017-01-02,8272,3984,544,1499
2,2017-01-03,6775,3814,501,1365
3,2017-01-04,7226,5297,460,1509
4,2017-01-05,7926,7513,468,1626


In [0]:
# Split out indivudal datasets for forecasting
phone = fact_table[['ds', 'phone']].rename(columns = {'phone': 'y'})
tablet = fact_table[['ds', 'tablet']].rename(columns = {'tablet': 'y'})
wearable = fact_table[['ds', 'wearable']].rename(columns = {'wearable': 'y'})
others = fact_table[['ds', 'others']].rename(columns = {'others': 'y'})

In [6]:
# Check the individual dataset format (testing only)
phone.head()

,ds,y
0,2017-01-01,3558
1,2017-01-02,8272
2,2017-01-03,6775
3,2017-01-04,7226
4,2017-01-05,7926


In [0]:
# Define the holidays
new_years_day = pd.DataFrame({
    'holiday': 'new_years_day',
    'ds': pd.to_datetime(['2017-01-01',
                          '2018-01-01',
                          '2019-01-01',
                          '2020-01-01']),
    'lower_window': 0,
    'upper_window': 0,
})

mlk_day = pd.DataFrame({
    'holiday': 'mlk_day',
    'ds': pd.to_datetime(['2017-01-16',
                          '2018-01-15',
                          '2019-01-21',
                          '2020-01-20']),
    'lower_window': 0,
    'upper_window': 0,
})

easter_sunday = pd.DataFrame({
    'holiday': 'easter_sunday',
    'ds': pd.to_datetime(['2017-04-16',
                          '2018-04-01',
                          '2019-04-21',
                          '2020-04-12']),
    'lower_window': 0,
    'upper_window': 0,
})

memorial_day = pd.DataFrame({
    'holiday': 'memorial_day',
    'ds': pd.to_datetime(['2017-05-29',
                          '2018-05-28',
                          '2019-05-27',
                          '2020-05-25']),
    'lower_window': 0,
    'upper_window': 0,
})

independence_day = pd.DataFrame({
    'holiday': 'independence_day',
    'ds': pd.to_datetime(['2017-07-04',
                          '2018-07-04',
                          '2019-07-04',
                          '2020-07-04']),
    'lower_window': 0,
    'upper_window': 0,
})

thanksgiving_day = pd.DataFrame({
    'holiday': 'thanksgiving_day',
    'ds': pd.to_datetime(['2017-11-23',
                          '2018-11-22',
                          '2019-11-28',
                          '2020-11-26']),
    'lower_window': -3,
    'upper_window': 0,
})

black_friday = pd.DataFrame({
    'holiday': 'black_friday',
    'ds': pd.to_datetime(['2017-11-24',
                          '2018-11-23',
                          '2019-11-29',
                          '2020-11-27']),
    'lower_window': 0,
    'upper_window': 1,
})

cyber_monday = pd.DataFrame({
    'holiday': 'cyber_monday',
    'ds': pd.to_datetime(['2017-11-27',
                          '2018-11-26',
                          '2019-12-02',
                          '2020-11-30']),
    'lower_window': 0,
    'upper_window': 1,
})

christmas_day = pd.DataFrame({
    'holiday': 'christmas_day',
    'ds': pd.to_datetime(['2017-12-25',
                          '2018-12-25',
                          '2019-12-25',
                          '2020-12-25']),
    'lower_window': -8,
    'upper_window': 3,
})

apple_phone_launch = pd.DataFrame({
    'holiday': 'apple_phone_launch',
    'ds': pd.to_datetime(['2017-09-22',
                          '2017-11-03',
                          '2018-09-21',
                          '2018-10-26',
                          '2019-09-20']),
    'lower_window': 0,
    'upper_window': 0,
})

samsung_phone_launch = pd.DataFrame({
    'holiday': 'samsung_phone_launch',
    'ds': pd.to_datetime(['2017-04-21',
                          '2018-03-16',
                          '2019-03-08',
                          '2020-03-06']),
    'lower_window': 0,
    'upper_window': 0,
})

samsung_note_launch = pd.DataFrame({
    'holiday': 'samsung_note_launch',
    'ds': pd.to_datetime(['2017-09-15',
                          '2018-08-24',
                          '2019-08-23']),
    'lower_window': 0,
    'upper_window': 0,
})

apple_tablet_launch = pd.DataFrame({
    'holiday': 'apple_tablet_launch',
    'ds': pd.to_datetime(['2017-03-24',
                          '2017-06-05',
                          '2018-03-27',
                          '2018-11-07',
                          '2019-03-18']),
    'lower_window': 0,
    'upper_window': 0,
})

samsung_tablet_launch = pd.DataFrame({
    'holiday': 'samsung_tablet_launch',
    'ds': pd.to_datetime(['2017-03-24',
                          '2018-08-10',
                          '2019-04-26',
                          '2019-08-30']),
    'lower_window': 0,
    'upper_window': 0,
})

apple_watch_launch = pd.DataFrame({
    'holiday': 'apple_watch_launch',
    'ds': pd.to_datetime(['2017-09-20',
                          '2018-09-19',
                          '2019-09-20']),
    'lower_window': 0,
    'upper_window': 0,
})

jan_bill_cycle = pd.DataFrame({
    'holiday': 'jan_bill_cycle',
    'ds': pd.to_datetime(['2018-01-05',
                          '2018-01-11',
                          '2018-01-14',
                          '2018-01-17',
                          '2018-01-24',
                          '2018-01-27',
                          '2019-01-29',
                          '2018-01-30',
                          '2018-01-31',
                          '2019-01-05',
                          '2019-01-11',
                          '2019-01-14',
                          '2019-01-17',
                          '2019-01-24',
                          '2019-01-27',
                          '2019-01-29',
                          '2019-01-30',
                          '2019-01-31',
                          '2020-01-05',
                          '2020-01-11',
                          '2020-01-14',
                          '2020-01-17',
                          '2020-01-24',
                          '2020-01-27',
                          '2020-01-29',
                          '2020-01-30',
                          '2020-01-31']),
    'lower_window': 0,
    'upper_window': 0,
})

feb_bill_cycle = pd.DataFrame({
    'holiday': 'feb_bill_cycle',
    'ds': pd.to_datetime(['2018-02-05',
                          '2018-02-11',
                          '2018-02-14',
                          '2018-02-17',
                          '2018-02-24',
                          '2018-02-27',
                          '2019-02-05',
                          '2019-02-11',
                          '2019-02-14',
                          '2019-02-17',
                          '2019-02-24',
                          '2019-02-27',
                          '2020-02-05',
                          '2020-02-11',
                          '2020-02-14',
                          '2020-02-17',
                          '2020-02-24',
                          '2020-02-27',
                          '2020-02-29']),
    'lower_window': 0,
    'upper_window': 0,
})

mar_bill_cycle = pd.DataFrame({
    'holiday': 'mar_bill_cycle',
    'ds': pd.to_datetime(['2018-03-05',
                          '2018-03-11',
                          '2018-03-14',
                          '2018-03-17',
                          '2018-03-24',
                          '2018-03-27',
                          '2019-03-29',
                          '2018-03-30',
                          '2018-03-31',
                          '2019-03-05',
                          '2019-03-11',
                          '2019-03-14',
                          '2019-03-17',
                          '2019-03-24',
                          '2019-03-27',
                          '2019-03-29',
                          '2019-03-30',
                          '2019-03-31',
                          '2020-03-05',
                          '2020-03-11',
                          '2020-03-14',
                          '2020-03-17',
                          '2020-03-24',
                          '2020-03-27',
                          '2020-03-29',
                          '2020-03-30',
                          '2020-03-31']),
    'lower_window': 0,
    'upper_window': 0,
})

apr_bill_cycle = pd.DataFrame({
    'holiday': 'apr_bill_cycle',
    'ds': pd.to_datetime(['2018-04-05',
                          '2018-04-11',
                          '2018-04-14',
                          '2018-04-17',
                          '2018-04-24',
                          '2018-04-27',
                          '2019-04-29',
                          '2018-04-30',
                          '2019-04-05',
                          '2019-04-11',
                          '2019-04-14',
                          '2019-04-17',
                          '2019-04-24',
                          '2019-04-27',
                          '2019-04-29',
                          '2019-04-30',
                          '2020-04-05',
                          '2020-04-11',
                          '2020-04-14',
                          '2020-04-17',
                          '2020-04-24',
                          '2020-04-27',
                          '2020-04-29',
                          '2020-04-30']),
    'lower_window': 0,
    'upper_window': 0,
})

may_bill_cycle = pd.DataFrame({
    'holiday': 'may_bill_cycle',
    'ds': pd.to_datetime(['2018-05-05',
                          '2018-05-11',
                          '2018-05-14',
                          '2018-05-17',
                          '2018-05-24',
                          '2018-05-27',
                          '2019-05-29',
                          '2018-05-30',
                          '2018-05-31',
                          '2019-05-05',
                          '2019-05-11',
                          '2019-05-14',
                          '2019-05-17',
                          '2019-05-24',
                          '2019-05-27',
                          '2019-05-29',
                          '2019-05-30',
                          '2019-05-31',
                          '2020-05-05',
                          '2020-05-11',
                          '2020-05-14',
                          '2020-05-17',
                          '2020-05-24',
                          '2020-05-27',
                          '2020-05-29',
                          '2020-05-30',
                          '2020-05-31']),
    'lower_window': 0,
    'upper_window': 0,
})

jun_bill_cycle = pd.DataFrame({
    'holiday': 'jun_bill_cycle',
    'ds': pd.to_datetime(['2018-06-05',
                          '2018-06-11',
                          '2018-06-14',
                          '2018-06-17',
                          '2018-06-24',
                          '2018-06-27',
                          '2019-06-29',
                          '2018-06-30',
                          '2019-06-05',
                          '2019-06-11',
                          '2019-06-14',
                          '2019-06-17',
                          '2019-06-24',
                          '2019-06-27',
                          '2019-06-29',
                          '2019-06-30',
                          '2020-06-05',
                          '2020-06-11',
                          '2020-06-14',
                          '2020-06-17',
                          '2020-06-24',
                          '2020-06-27',
                          '2020-06-29',
                          '2020-06-30']),
    'lower_window': 0,
    'upper_window': 0,
})

jul_bill_cycle = pd.DataFrame({
    'holiday': 'jul_bill_cycle',
    'ds': pd.to_datetime(['2018-07-05',
                          '2018-07-11',
                          '2018-07-14',
                          '2018-07-17',
                          '2018-07-24',
                          '2018-07-27',
                          '2019-07-29',
                          '2018-07-30',
                          '2018-07-31',
                          '2019-07-05',
                          '2019-07-11',
                          '2019-07-14',
                          '2019-07-17',
                          '2019-07-24',
                          '2019-07-27',
                          '2019-07-29',
                          '2019-07-30',
                          '2019-07-31',
                          '2020-07-05',
                          '2020-07-11',
                          '2020-07-14',
                          '2020-07-17',
                          '2020-07-24',
                          '2020-07-27',
                          '2020-07-29',
                          '2020-07-30',
                          '2020-07-31']),
    'lower_window': 0,
    'upper_window': 0,
})

aug_bill_cycle = pd.DataFrame({
    'holiday': 'aug_bill_cycle',
    'ds': pd.to_datetime(['2018-08-05',
                          '2018-08-11',
                          '2018-08-14',
                          '2018-08-17',
                          '2018-08-24',
                          '2018-08-27',
                          '2019-08-29',
                          '2018-08-30',
                          '2018-08-31',
                          '2019-08-05',
                          '2019-08-11',
                          '2019-08-14',
                          '2019-08-17',
                          '2019-08-24',
                          '2019-08-27',
                          '2019-08-29',
                          '2019-08-30',
                          '2019-08-31',
                          '2020-08-05',
                          '2020-08-11',
                          '2020-08-14',
                          '2020-08-17',
                          '2020-08-24',
                          '2020-08-27',
                          '2020-08-29',
                          '2020-08-30',
                          '2020-08-31']),
    'lower_window': 0,
    'upper_window': 0,
})

sep_bill_cycle = pd.DataFrame({
    'holiday': 'sep_bill_cycle',
    'ds': pd.to_datetime(['2018-09-05',
                          '2018-09-11',
                          '2018-09-14',
                          '2018-09-17',
                          '2018-09-24',
                          '2018-09-27',
                          '2019-09-29',
                          '2018-09-30',
                          '2019-09-05',
                          '2019-09-11',
                          '2019-09-14',
                          '2019-09-17',
                          '2019-09-24',
                          '2019-09-27',
                          '2019-09-29',
                          '2019-09-30',
                          '2020-09-05',
                          '2020-09-11',
                          '2020-09-14',
                          '2020-09-17',
                          '2020-09-24',
                          '2020-09-27',
                          '2020-09-29',
                          '2020-09-30']),
    'lower_window': 0,
    'upper_window': 0,
})

oct_bill_cycle = pd.DataFrame({
    'holiday': 'oct_bill_cycle',
    'ds': pd.to_datetime(['2018-10-05',
                          '2018-10-11',
                          '2018-10-14',
                          '2018-10-17',
                          '2018-10-24',
                          '2018-10-27',
                          '2019-10-29',
                          '2018-10-30',
                          '2018-10-31',
                          '2019-10-05',
                          '2019-10-11',
                          '2019-10-14',
                          '2019-10-17',
                          '2019-10-24',
                          '2019-10-27',
                          '2019-10-29',
                          '2019-10-30',
                          '2019-10-31',
                          '2020-10-05',
                          '2020-10-11',
                          '2020-10-14',
                          '2020-10-17',
                          '2020-10-24',
                          '2020-10-27',
                          '2020-10-29',
                          '2020-10-30',
                          '2020-10-31']),
    'lower_window': 0,
    'upper_window': 0,
})

nov_bill_cycle = pd.DataFrame({
    'holiday': 'nov_bill_cycle',
    'ds': pd.to_datetime(['2018-11-05',
                          '2018-11-11',
                          '2018-11-14',
                          '2018-11-17',
                          '2018-11-24',
                          '2018-11-27',
                          '2019-11-29',
                          '2018-11-30',
                          '2019-11-05',
                          '2019-11-11',
                          '2019-11-14',
                          '2019-11-17',
                          '2019-11-24',
                          '2019-11-27',
                          '2019-11-29',
                          '2019-11-30',
                          '2020-11-05',
                          '2020-11-11',
                          '2020-11-14',
                          '2020-11-17',
                          '2020-11-24',
                          '2020-11-27',
                          '2020-11-29',
                          '2020-11-30']),
    'lower_window': 0,
    'upper_window': 0,
})

dec_bill_cycle = pd.DataFrame({
    'holiday': 'dec_bill_cycle',
    'ds': pd.to_datetime(['2018-12-05',
                          '2018-12-11',
                          '2018-12-14',
                          '2018-12-17',
                          '2018-12-24',
                          '2018-12-27',
                          '2019-12-29',
                          '2018-12-30',
                          '2018-12-31',
                          '2019-12-05',
                          '2019-12-11',
                          '2019-12-14',
                          '2019-12-17',
                          '2019-12-24',
                          '2019-12-27',
                          '2019-12-29',
                          '2019-12-30',
                          '2019-12-31',
                          '2020-12-05',
                          '2020-12-11',
                          '2020-12-14',
                          '2020-12-17',
                          '2020-12-24',
                          '2020-12-27',
                          '2020-12-29',
                          '2020-12-30',
                          '2020-12-31']),
    'lower_window': 0,
    'upper_window': 0,
})

In [0]:
holidays_phone = pd.concat((new_years_day,
                            mlk_day,
                            easter_sunday,
                            memorial_day,
                            independence_day,
                            thanksgiving_day,
                            black_friday,
                            cyber_monday,
                            christmas_day,
                            apple_phone_launch,
                            samsung_phone_launch,
                            samsung_note_launch,
                            jan_bill_cycle,
                            feb_bill_cycle,
                            mar_bill_cycle,
                            apr_bill_cycle,
                            may_bill_cycle,
                            jun_bill_cycle,
                            jul_bill_cycle,
                            aug_bill_cycle,
                            sep_bill_cycle,
                            oct_bill_cycle,
                            nov_bill_cycle,
                            dec_bill_cycle))

In [0]:
holidays_tablet = pd.concat((new_years_day,
                             mlk_day,
                             easter_sunday,
                             memorial_day,
                             independence_day,
                             thanksgiving_day,
                             black_friday,
                             cyber_monday,
                             christmas_day,
                             apple_tablet_launch,
                             samsung_tablet_launch,
                             jan_bill_cycle,
                             feb_bill_cycle,
                             mar_bill_cycle,
                             apr_bill_cycle,
                             may_bill_cycle,
                             jun_bill_cycle,
                             jul_bill_cycle,
                             aug_bill_cycle,
                             sep_bill_cycle,
                             oct_bill_cycle,
                             nov_bill_cycle,
                             dec_bill_cycle))

In [0]:
holidays_wearable = pd.concat((new_years_day,
                               mlk_day,
                               easter_sunday,
                               memorial_day,
                               independence_day,
                               thanksgiving_day,
                               black_friday,
                               cyber_monday,
                               christmas_day,
                               apple_watch_launch,
                               jan_bill_cycle,
                               feb_bill_cycle,
                               mar_bill_cycle,
                               apr_bill_cycle,
                               may_bill_cycle,
                               jun_bill_cycle,
                               jul_bill_cycle,
                               aug_bill_cycle,
                               sep_bill_cycle,
                               oct_bill_cycle,
                               nov_bill_cycle,
                               dec_bill_cycle))

In [0]:
holidays_others = pd.concat((new_years_day,
                             mlk_day,
                             easter_sunday,
                             memorial_day,
                             independence_day,
                             thanksgiving_day,
                             black_friday,
                             cyber_monday,
                             christmas_day,
                             apple_watch_launch,
                             jan_bill_cycle,
                             feb_bill_cycle,
                             mar_bill_cycle,
                             apr_bill_cycle,
                             may_bill_cycle,
                             jun_bill_cycle,
                             jul_bill_cycle,
                             aug_bill_cycle,
                             sep_bill_cycle,
                             oct_bill_cycle,
                             nov_bill_cycle,
                             dec_bill_cycle))

In [12]:
# Define and fit the models
phone_model = Prophet(holidays = holidays_phone).fit(phone)
tablet_model = Prophet(holidays = holidays_tablet).fit(tablet)
wearable_model = Prophet(holidays = holidays_wearable).fit(wearable)
others_model = Prophet(holidays = holidays_others).fit(others)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [0]:
# NUmber of periods into the future to forecast
n = 31

In [0]:
# Create dataframes to hold predictions
phone_future = phone_model.make_future_dataframe(periods = n)
tablet_future = tablet_model.make_future_dataframe(periods = n)
wearable_future = wearable_model.make_future_dataframe(periods = n)
others_future = others_model.make_future_dataframe(periods = n)

In [0]:
# Make predictions
phone_forecast = phone_model.predict(phone_future)
tablet_forecast = tablet_model.predict(tablet_future)
wearable_forecast = wearable_model.predict(wearable_future)
others_forecast = others_model.predict(others_future)

In [0]:
# Get the datestamp and prediction columns
phone_forecast_subset = phone_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Phone'})
tablet_forecast_subset = tablet_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Tablet'})
wearable_forecast_subset = wearable_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Wearable'})
others_forecast_subset = others_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Others'})

In [17]:
# Check the format of the subsetted forecasts (testing only)
phone_forecast_subset.tail()

,ds,Phone
1181,2020-03-27,10986.842583
1182,2020-03-28,10806.751934
1183,2020-03-29,7718.671212
1184,2020-03-30,9064.510838
1185,2020-03-31,9477.580487


In [0]:
# Combine the forecasts into one table
aal_pred_table = phone_forecast_subset
aal_pred_table.insert(2, 'Tablet', tablet_forecast_subset['Tablet'])
aal_pred_table.insert(3, 'Wearable', wearable_forecast_subset['Wearable'])
aal_pred_table.insert(4, 'Others', others_forecast_subset['Others'])

In [19]:
# Check the format of the final predictions table (testing only)
aal_pred_table.tail()

,ds,Phone,Tablet,Wearable,Others
1181,2020-03-27,10986.842583,4005.010552,4048.861603,1684.021780
1182,2020-03-28,10806.751934,3187.635613,3446.159083,1553.643392
1183,2020-03-29,7718.671212,1561.486888,2021.078102,512.968474
1184,2020-03-30,9064.510838,2739.754392,2768.807044,1226.596063
1185,2020-03-31,9477.580487,2983.238048,3200.950120,1294.215476


In [0]:
# Save the predictions table to a csv file
aal_pred_table.to_csv('drive/My Drive/AAL/aal_pred_table.csv')